In [1]:
import os
import numpy as np
import torch
import tqdm


In [2]:
root = "data_prepared/train"

all_videos = [name for name in os.listdir(root)
            if os.path.isdir(os.path.join(root, name))]
all_videos.sort(key= lambda x: int(x.split('/')[-1].split('.')[0].split('_')[-1]))


In [3]:
mask_sample = torch.load(os.path.join(root, all_videos[0], 'mask.pt')).numpy()


In [4]:
# fp = np.memmap(os.path.join(root, 'masks.bin'), dtype='float16', mode='w+', shape=(len(all_videos), *mask_sample.shape))
# for i, video in tqdm.tqdm(enumerate(all_videos), total=len(all_videos)):
#     mask = torch.load(os.path.join(root, video, 'mask.pt')).numpy()
#     fp[i] = mask
#     fp.flush()


In [10]:
class MemMapDataset(torch.utils.data.Dataset):
    def __init__(self, root, split, length, t=22, c=49, h=160, w=240):
        path = os.path.join(root, split, 'masks.bin')
        self.fp = np.memmap(path, dtype='float16', mode='r', shape=(length, t, c, h, w))

    def __getitem__(self, index):
        return torch.from_numpy(self.fp[index])

    def __len__(self):
        return len(self.fp)


In [11]:
class MaskPredDataset(torch.utils.data.Dataset):
    def __init__(self, data_root, split):
        self.data_path = os.path.join(data_root, split)
        self.split = split

        all_videos = [name for name in os.listdir(root)
            if os.path.isdir(os.path.join(root, name))]
        all_videos.sort(key= lambda x: int(x.split('/')[-1].split('.')[0].split('_')[-1]))
        self.all_videos = all_videos

        # all_masks = []
        # for video in tqdm.tqdm(all_videos, postfix="Loading masks for split {}".format(split)):
        #     mask_path = os.path.join(self.data_path, video, "mask.pt")
        #     mask = torch.load(mask_path)
        #     all_masks.append(mask.float())
        # self.all_masks = all_masks

    def __len__(self):
        return len(self.all_videos)

    def __getitem__(self, idx):
        mask_path = os.path.join(self.data_path, self.all_videos[idx], "mask.pt")
        mask = torch.load(mask_path)
        return mask.float()
        # return self.all_masks[idx]


In [12]:
dataset_mm = MemMapDataset(os.path.join(root, 'masks.bin'))
dataloader_mm = torch.utils.data.DataLoader(dataset_mm, batch_size=4, shuffle=False, num_workers=4)

dataset = MaskPredDataset('data_prepared', 'train')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False, num_workers=4)


In [ ]:
for mask in tqdm.tqdm(dataloader_mm):
    print(mask.shape)
    mask = mask.to('cuda')
    print(mask.shape)
    break


In [8]:
for mask in tqdm.tqdm(dataloader):
    pass


100%|██████████| 250/250 [01:52<00:00,  2.23it/s]


In [ ]:
# Memmap Dataset one epoch read time
# 2.27 normal
# 1.16-1.08 with bs 4
# 0.22 with torch.from_numpy

# Normal Dataset one epoch read time
# 1:52 with bs 4
